<a href="https://colab.research.google.com/github/Jamal811/Generative-AI/blob/main/automatic_grader_example_(gimini_api_and_gemini_1_5_flash_model).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install langchain

In [2]:
!pip install langchain-community

In [3]:
!pip install google-generativeai langchain-google-genai

In [4]:
!pip install -qU langchain-google-genai

In [5]:
import getpass
import os

if "GOOGLE_API_KEY" not in os.environ:
    os.environ["GOOGLE_API_KEY"] = getpass.getpass("Enter your Google AI API key: ")

Enter your Google AI API key: ··········


In [6]:
from langchain_google_genai import ChatGoogleGenerativeAI

llm = ChatGoogleGenerativeAI(
    model="gemini-1.5-pro",
    temperature=0,
    max_tokens=None,
    timeout=None,
    max_retries=2,
    # other params...
)

In [8]:
from google.colab import userdata
api_key = userdata.get('key')  # It will show a secure input form

In [9]:
# Import libraries
from langchain_google_genai import ChatGoogleGenerativeAI
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from langchain.schema import BaseOutputParser

In [20]:
chat_model = ChatGoogleGenerativeAI(model="gemini-1.5-flash",temperature=0.7)

In [21]:
# Define prompt template
prompt_template_text = """
You are a high school history teacher grading homework assignments.
Based on the homework question indicated by “**Q:**” and the correct answer indicated by “**A:**”, your task is to determine whether the student's answer is correct.
Grading is binary; therefore, student answers can be correct or wrong.
Simple misspellings are okay.

**Q:** {question}
**A:** {correct_answer}

**Student's Answer:** {student_answer}
"""

In [22]:
prompt = PromptTemplate(input_variables=["question", "correct_answer", "student_answer"], template=prompt_template_text)

# Define chain
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
)

In [23]:
import time

# Define inputs
question = "Who was the 35th president of the United States of America?"
correct_answer = "John F. Kennedy"
student_answer_list = ["John F. Kennedy", "JFK", "FDR", "John F. Kenedy", "John Kennedy", "Jack Kennedy", "Jacqueline Kennedy", "Robert F. Kenedy"]

# Run chain in a loop for multiple student answers
# with added delay
for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question': question, 'correct_answer': correct_answer, 'student_answer': student_answer})))
    print('\n')
    time.sleep(2) # Wait for 2 seconds before the next request

John F. Kennedy - Correct 



JFK - Correct. 



FDR - Wrong. 



John F. Kenedy - The student's answer is **wrong**. While the student has the correct president, the spelling of the last name is incorrect. 



John Kennedy - The student's answer is **correct**.  While it's missing the middle initial, the answer is still clearly referring to John F. Kennedy. 



Jack Kennedy - The student's answer is **correct**. 

While "John F. Kennedy" is the full, formal name, "Jack Kennedy" is a widely recognized nickname for the 35th president. 



Jacqueline Kennedy - Wrong. 



Robert F. Kenedy - The student's answer is **wrong**. 

The correct answer is John F. Kennedy, not Robert F. Kennedy. While both were important figures in American history, John F. Kennedy was the 35th president. 





In [24]:
# Define output parser
class GradeOutputParser(BaseOutputParser):
    """Determine whether grade was correct or wrong"""

    def parse(self, text: str):
        """Parse the output of an LLM call."""
        return "wrong" not in text.lower()

# Update chain with output parser
chain = LLMChain(
    llm=chat_model,
    prompt=prompt,
    output_parser=GradeOutputParser()
)

In [25]:
# Grade student answers
for student_answer in student_answer_list:
    print(student_answer + " - " + str(chain.run({'question': question, 'correct_answer': correct_answer, 'student_answer': student_answer})))

John F. Kennedy - True
JFK - True
FDR - False
John F. Kenedy - False
John Kennedy - True
Jack Kennedy - True
Jacqueline Kennedy - False
Robert F. Kenedy - False
